In [2]:
from __future__ import division, print_function, unicode_literals, \
    absolute_import

"""
Wulff construction to create the nanoparticle
"""

from six.moves import range

import itertools
from math import gcd
from functools import reduce

import numpy as np

from pymatgen.core.structure import Structure, Molecule
from pymatgen.symmetry.analyzer import SpacegroupAnalyzer
from pymatgen.util.coord import in_coord_list
from pymatgen import MPRester
MAPI_KEY='4oBTKz0pkFSg9EUQ'

rester = MPRester(MAPI_KEY)
from mpinterfaces import get_struct_from_mp


#def get_struct_from_mp(mp):
    

#from mpinterfaces.default_logger import get_default_logger

#logger = get_default_logger(__name__)


class Nanoparticle(Molecule):
    """
    Construct nanoparticle using wulff construction
    """

    def __init__(self, structure, rmax=15, hkl_family=((1, 0, 0), (1, 1, 1)),
                 surface_energies=(28, 25)):
        self.structure = structure
        self.rmax = rmax
        self.hkl_family = list(hkl_family)
        self.surface_energies = list(surface_energies)
        spherical_neighbors = self.structure.get_sites_in_sphere(
            [0.0, 0.0, 0.0], self.rmax)
        recp_lattice = self.structure.lattice.reciprocal_lattice_crystallographic
        self.recp_lattice = recp_lattice.scale(1)
        self.set_miller_family()
        Molecule.__init__(self, [sn[0].species_and_occu
                                 for sn in spherical_neighbors],
                          [sn[0].coords for sn in spherical_neighbors],
                          charge=0)

    def set_miller_family(self):
        """
        get all miller indices for the given maximum index
        get the list of indices that correspond to the given family
        of indices
        """
        recp_structure = Structure(self.recp_lattice, ["H"], [[0, 0, 0]])
        analyzer = SpacegroupAnalyzer(recp_structure, symprec=0.001)
        symm_ops = analyzer.get_symmetry_operations()

        max_index = max(max(m) for m in self.hkl_family)
        r = list(range(-max_index, max_index + 1))
        r.reverse()
        miller_indices = []
        self.all_equiv_millers = []
        self.all_surface_energies = []
        for miller in itertools.product(r, r, r):
            if any([i != 0 for i in miller]):
                d = abs(reduce(gcd, miller))
                miller_index = tuple([int(i / d) for i in miller])
                for op in symm_ops:
                    for i, u_miller in enumerate(self.hkl_family):
                        if in_coord_list(u_miller, op.operate(miller_index)):
                            self.all_equiv_millers.append(miller_index)
                            self.all_surface_energies.append(
                                self.surface_energies[i])

    def get_normals(self):
        """
        get the normal to the plane (h,k,l)
        """
        normals = []
        for hkl in self.all_equiv_millers:
            normal = self.recp_lattice.matrix[0, :] * hkl[0] + \
                self.recp_lattice.matrix[1, :] * hkl[1] + \
                self.recp_lattice.matrix[2, :] * hkl[2]
            normals.append(normal / np.linalg.norm(normal))
        return normals

    def get_centered_molecule(self):
        center = self.center_of_mass
        new_coords = np.array(self.cart_coords) - center
        return Molecule(self.species_and_occu, new_coords,
                        charge=self._charge,
                        spin_multiplicity=self._spin_multiplicity,
                        site_properties=self.site_properties)

    def create(self):
        """
        creates the nanoparticle by chopping of the corners normal to the
        specified surfaces.
        the distance to the surface from the center of the particel =
        normalized surface energy * max radius
        """
        mol = self.get_centered_molecule()
        normalized_surface_energies = \
            np.array(self.all_surface_energies) / float(
                max(self.all_surface_energies))
        surface_normals = self.get_normals()
        remove_sites = []
        for i, site in enumerate(mol):
            for j, normal in enumerate(surface_normals):
                n = np.array(normal)
                n = n / np.linalg.norm(n)
                if np.dot(site.coords, n) + self.rmax * \
                        normalized_surface_energies[j] <= 0:
                    remove_sites.append(i)
                    break
        self.remove_sites(remove_sites)
        # new_sites = [site for k, site in enumerate(mol) if k not in remove_sites]
        # return Molecule.from_sites(new_sites)



    """
    Wulff construction using the ASE package
    works only for cubic systems and doesn't support multiatom basis
    from ase.cluster import wulff_construction
    from pymatgen.io.aseio import AseAtomsAdaptor
    symbol = 'Pt'
    surfaces = [ (1,0,0), (1,1,1) ]
    surface_energies = [1, 1]
    size = 200 #number of atoms
    structure = "fcc"
    latticeconstant = 5.0
    atoms = wulff_construction(symbol, surfaces, surface_energies, size, structure,
    rounding='closest', latticeconstant=latticeconstant,
    debug=False, maxiter=100)
    #convert to pymatgen structure
    pgen_structure = AseAtomsAdaptor().get_structure(atoms)
    pgen_structure.to(fmt='poscar', filename='POSCAR_pt_nano.vasp')
    """

/home/fgimbert/anaconda3/envs/mlenv/lib/python3.7/site-packages/MPInterfaces-2018.3.21-py3.7.egg/mpinterfaces/__init__.py:28: UserWarning: mpint_config.yaml file not configured.
  warnings.warn('mpint_config.yaml file not configured.')


In [3]:
structure = rester.get_structure_by_material_id('mp-33')


In [4]:
rmax = 25
    # surface families to be chopped off
surface_families = [(1, 0, 0), (1, 0, 1)]
surface_families = [(1,1,1), (1,0,0), (1,1,0)]
surface_energies = [18, 20, 20]
# could be in any units, will be normalized
#surface_energies = [28, 25]

# caution: set the structure wrt which the the miller indices are specified
# use your own API key
# primitve ---> conventional cell
sa = SpacegroupAnalyzer(structure)
structure_conventional = sa.get_conventional_standard_structure()

nanoparticle = Nanoparticle(structure_conventional, rmax=rmax,
                                hkl_family=surface_families,
                                surface_energies=surface_energies)
nanoparticle.create()

/home/fgimbert/anaconda3/envs/mlenv/lib/python3.7/site-packages/ipykernel_launcher.py:51: DeprecationWarning: species_and_occu is deprecated
Use site.species instead. This will be deprecated with effect from pymatgen 2020.
/home/fgimbert/anaconda3/envs/mlenv/lib/python3.7/site-packages/ipykernel_launcher.py:51: DeprecationWarning: species_and_occu is deprecated
Use site.species instead. This will be deprecated with effect from pymatgen 2020.
/home/fgimbert/anaconda3/envs/mlenv/lib/python3.7/site-packages/ipykernel_launcher.py:51: DeprecationWarning: species_and_occu is deprecated
Use site.species instead. This will be deprecated with effect from pymatgen 2020.
/home/fgimbert/anaconda3/envs/mlenv/lib/python3.7/site-packages/ipykernel_launcher.py:51: DeprecationWarning: species_and_occu is deprecated
Use site.species instead. This will be deprecated with effect from pymatgen 2020.
/home/fgimbert/anaconda3/envs/mlenv/lib/python3.7/site-packages/ipykernel_launcher.py:51: DeprecationWarning

In [5]:
structure

Structure Summary
Lattice
    abc : 2.73881799 2.738817996239641 4.29205455
 angles : 90.0 90.0 120.00000004541803
 volume : 27.88188998539158
      A : 2.73881799 0.0 0.0
      B : -1.369409 2.37188596 0.0
      C : 0.0 0.0 4.29205455
PeriodicSite: Ru (-0.0000, 1.5813, 1.0730) [0.3333, 0.6667, 0.2500]
PeriodicSite: Ru (1.3694, 0.7906, 3.2190) [0.6667, 0.3333, 0.7500]

In [6]:
nanoparticle.to(fmt='xyz', filename='nanoparticle.xyz')

/home/fgimbert/anaconda3/envs/mlenv/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


'4514\nRu4514\nRu -1.369409 -24.509488 -1.073014\nRu -1.369409 -22.928231 -3.219041\nRu -1.369409 -22.928231 -7.511095\nRu -1.369409 -19.765716 -1.073014\nRu -1.369409 -18.184459 -3.219041\nRu -1.369409 -19.765716 -5.365068\nRu -1.369409 -18.184459 -7.511095\nRu -1.369409 -19.765716 -9.657123\nRu -1.369409 -18.184459 -11.803150\nRu -1.369409 -19.765716 -13.949177\nRu -1.369409 -18.184459 -16.095205\nRu -2.738818 -22.137602 -1.073014\nRu -2.738818 -20.556345 -3.219041\nRu -2.738818 -22.137602 -5.365068\nRu -2.738818 -20.556345 -7.511095\nRu -2.738818 -22.137602 -9.657123\nRu -2.738818 -20.556345 -11.803150\nRu -4.108227 -24.509488 -1.073014\nRu -4.108227 -22.928231 -3.219041\nRu -4.108227 -22.928231 -7.511095\nRu -1.369409 -15.021944 -1.073014\nRu -1.369409 -13.440687 -3.219041\nRu -1.369409 -15.021944 -5.365068\nRu -1.369409 -13.440687 -7.511095\nRu -1.369409 -15.021944 -9.657123\nRu -1.369409 -13.440687 -11.803150\nRu -1.369409 -15.021944 -13.949177\nRu -1.369409 -13.440687 -16.095205

In [7]:
from ase.visualize import view
from ase.io import read, write
atoms = read('nanoparticle.xyz')

In [8]:
view(atoms, viewer='ngl')

_ColormakerRegistry()

/home/fgimbert/anaconda3/envs/mlenv/lib/python3.7/site-packages/nglview/widget.py:162: DeprecationWarning: Traits should be given as instances, not types (for example, `Int()`, not `Int`). Passing types is deprecated in traitlets 4.1.
  _ngl_view_id = List(Unicode).tag(sync=True)


In [146]:
from ase.build import bulk
from ase.calculators.espresso import Espresso
from ase.constraints import UnitCellFilter
from ase.optimize import LBFGS
import os 
pseudopotentials = {'Na': 'Na.pbe-spnl-rrkjus_psl.1.0.0.UPF',
                    'Cl': 'Cl.pbe-nl-rrkjus_psl.1.0.0.UPF'}

os.environ['ESPRESSO_PSEUDO'] = "/home/fgimbert/Projects/SSSP_efficiency_pseudos"
os.environ['ASE_ESPRESSO_COMMAND'] = "/home/fgimbert/q-e-qe-6.5/bin/pw.x -in PREFIX.pwi > PREFIX.pwo"



rocksalt = bulk('NaCl', crystalstructure='rocksalt', a=6.0)
calc = Espresso(pseudopotentials=pseudopotentials,tstress=True, tprnfor=True, kpts=(3, 3, 3))
rocksalt.calc = calc
calc.write_input(rocksalt)


ucf = UnitCellFilter(rocksalt)
opt = LBFGS(ucf)
opt.run(fmax=0.005)

# cubic lattic constant
print((8*rocksalt.get_volume()/len(rocksalt))**(1.0/3.0))

CalculationFailed: Calculator "espresso" failed with command "/home/fgimbert/q-e-qe-6.5/bin/pw.x -in espresso.pwi > espresso.pwo" failed in /home/fgimbert/Projects/ASAPv1 with error code 1

In [147]:
path =  "/home/fgimbert/Projects/SSSP_efficiency_pseudos/"
            
            
list_files = [x for x in os.walk(path)][0][2]

In [148]:
pseudopot = {}
for file in list_files:
    pseudopot[file.split('_')[0]] = 0

In [115]:
pseudopot

{'Ag': 0,
 'Al.pbe-n-kjpaw': 0,
 'Ar': 0,
 'As.pbe-n-rrkjus': 0,
 'Au': 0,
 'Ba.pbe-spn-kjpaw': 0,
 'Bi': 0,
 'C.pbe-n-kjpaw': 0,
 'Ca': 0,
 'Cd.pbe-dn-rrkjus': 0,
 'Ce.GGA-PBE-paw-v1.0.UPF': 0,
 'Co': 0,
 'Cs': 0,
 'Cu': 0,
 'Dy.GGA-PBE-paw-v1.0.UPF': 0,
 'Er.GGA-PBE-paw-v1.0.UPF': 0,
 'Eu.GGA-PBE-paw-v1.0.UPF': 0,
 'Fe.pbe-spn-kjpaw': 0,
 'Ga.pbe-dn-kjpaw': 0,
 'Gd.GGA-PBE-paw-v1.0.UPF': 0,
 'H.pbe-rrkjus': 0,
 'He': 0,
 'Hf-sp.oncvpsp.upf': 0,
 'Hg': 0,
 'Ho.GGA-PBE-paw-v1.0.UPF': 0,
 'I.pbe-n-kjpaw': 0,
 'In.pbe-dn-rrkjus': 0,
 'Ir': 0,
 'K.pbe-spn-kjpaw': 0,
 'Kr': 0,
 'La.GGA-PBE-paw-v1.0.UPF': 0,
 'Lu.GGA-PBE-paw-v1.0.UPF': 0,
 'Mg.pbe-n-kjpaw': 0,
 'Mo': 0,
 'N.pbe-n-radius': 0,
 'Nb.pbe-spn-kjpaw': 0,
 'Nd.GGA-PBE-paw-v1.0.UPF': 0,
 'Ne': 0,
 'O.pbe-n-kjpaw': 0,
 'Os': 0,
 'P.pbe-n-rrkjus': 0,
 'Pb.pbe-dn-kjpaw': 0,
 'Pd': 0,
 'Pm.GGA-PBE-paw-v1.0.UPF': 0,
 'Po.pbe-dn-rrkjus': 0,
 'Pr.GGA-PBE-paw-v1.0.UPF': 0,
 'Rb': 0,
 'Re': 0,
 'Rh': 0,
 'Rn.pbe-dn-kjpaw': 0,
 'Ru': 0,
 'Sc

In [ ]:
from ase.build import bulk
from ase.calculators.vasp import Vasp2

import os

os.environ['VASP_PP_PATH'] = 'home/share/potpaw' 
os.environ['ASE_VASP_COMMAND'] = "mpirun -genv I_MPI_FABRICS=shm:dapl -genv I_MPI_DEBUG=5 -genv I_MPI_PIN=1 -genv I_MPI_PIN_MODE=pm -np 40 /home/share/vasp.5.4.4/bin/vasp_std"

print('Bonjour Ici asepy.py !')

si = bulk('Si')

mydir = 'Test'    # Directory where we will do the calculations

# Make self-consistent ground state
calc = Vasp2(kpts=(4, 4, 4), directory=mydir)

si.set_calculator(calc)
si.get_potential_energy()  # Run the calculation

# Non-SC calculation along band path
kpts = {'path': 'WGX',     # The BS path
        'npoints': 30}     # Number of points along the path

calc.set(isym=0,           # Turn off kpoint symmetry reduction
         icharg=11,        # Non-SC calculation
         kpts=kpts)

# Run the calculation
si.get_potential_energy()

